# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [3]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [4]:
earthquake_json['metadata']

{'generated': 1701276932000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-10-29&endtime=2023-11-28',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 9623}

Each element in the JSON array `features` is a row of data for our dataframe.

In [5]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [6]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.6,
  'place': '71 km N of Petersville, Alaska',
  'time': 1701129482633,
  'updated': 1701129611840,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023f7sxa3r',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023f7sxa3r&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 39,
  'net': 'ak',
  'code': '023f7sxa3r',
  'ids': ',ak023f7sxa3r,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.54,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.6 - 71 km N of Petersville, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-150.5493, 63.1338, 89.2]},
 'id': 'ak023f7sxa3r'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [7]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.60,"71 km N of Petersville, Alaska",1701129482633,1701129611840,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023f7sxa3r,",",ak,",",origin,phase-data,",NaN,NaN,0.54,NaN,ml,earthquake,"M 1.6 - 71 km N of Petersville, Alaska"
1,1.50,None,1701129458093,1701131936693,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",tx2023xgtl,",",tx,",",origin,phase-data,",15.0,0.100000,0.50,70.0,mlv,earthquake,M 1.5 -
2,1.50,None,1701129376338,1701188856320,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",tx2023xgti,",",tx,",",origin,phase-data,",19.0,0.000000,0.20,56.0,ml(texnet),earthquake,M 1.5 -
3,1.30,"55 km S of Whites City, New Mexico",1701129349709,1701187133125,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",tx2023xgtj,",",tx,",",origin,phase-data,",15.0,0.000000,0.20,58.0,ml(texnet),earthquake,"M 1.3 - 55 km S of Whites City, New Mexico"
4,0.24,"10 km NW of The Geysers, CA",1701128988560,1701129974473,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73968791,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",12.0,0.000293,0.02,76.0,md,earthquake,"M 0.2 - 10 km NW of The Geysers, CA"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>